In [6]:
%run Market.ipynb
%run Policy_Loader.ipynb
%run PolicyCons.ipynb

max_space_in_strong_market=100
max_space_in_unknown_market=50
max_space_in_weak_market=30
max_space_in_single_stock=10


class Risk:
    
    def __init__(self,account,symbol,context,policy_name):
        self.account = account
        self.symbol = symbol
        self.context = context
        
        self.short = context['short']
        self.mid = context['mid']
        self.policy_name = policy_name
        
        self.policy_loader = Policy_Loader(self.policy_name)
        self.market = Market(context)
        self.pCons = PolicyCons()
    
    def get_fix_stop_loss_percent(self):
        return self.policy_loader.get_stop_loss_fixed_percent()

        
#   牛市总仓位不超过80%
#   猴市总仓位不超过40%
#   熊市总仓位不超过20%
#   单笔仓位不超过30%
    def get_market_space(self,cur_date):
        market_status = self.market.get_market_status(cur_date)
        
        return self.policy_loader.get_one_account_position(market_status)


    def get_max_space(self,cur_date):
#         没有开启仓位控制，就全仓
        if(self.policy_loader.enable_one_account_position()==False):
            return max_space_in_strong_market
        else:
            market_space = self.get_market_space(cur_date)
            cur_space = self.account.get_space_per(cur_date)

            if(cur_space>=market_space):
                return 0
            else:
                return market_space-cur_space

        
    # 止损价格
    def get_stop_loss_price(self,price):
        if(self.policy_loader.enabled_stop_loss()==False):
            return None
            
        if(self.policy_loader.get_stop_loss_approach()==self.pCons.stop_loss_approach_fixed_percent):
            if(price>0):
                return price * (1-self.get_fix_stop_loss_percent()/100)
            else:
                return price * (1+self.get_fix_stop_loss_percent()/100)
        elif(self.policy_loader.get_stop_loss_approach()==self.pCons.stop_loss_approach_ATR):
#             TODO
            return None
        

        
#   不考虑仓位，单笔交易承担亏损总资金的3%
    def get_shares_only_care_loss_money(self,cur_price,risk_percent=2):
        risk_money = self.account.get_all_cash()*risk_percent/100
        
        #止损价格
        stop_loss_price = self.get_stop_loss_price(cur_price)
        plan_shares = risk_money/(cur_price-stop_loss_price)
        
        if(plan_shares<=0):
            print('account_money:'+str(account_money) + ' cur_price:'+str(cur_price)+ ' plan_shares:'+str(plan_shares))

        shares = int((plan_shares)/100)*100
        return shares
    
    
    def get_shares_only_care_about_space(self, cur_date, cur_price):
        max_space = self.get_max_space(cur_date)
        
        if(self.account.get_space_per(cur_date)>=max_space):
            return 0
        
        
        today_stock_asset = self.account.get_today_stock_asset(cur_date)
        today_cash = self.account.get_all_cash()
        
        to_buy_stock_money = (today_stock_asset+today_cash)*max_space/100
        
        if(to_buy_stock_money>today_cash):
            to_buy_stock_money = today_cash

        cur_price = round(cur_price,3)
        if(cur_price<0.001):
            return 0
        
        from_space_shares = int((to_buy_stock_money/cur_price)/100)*100

        return from_space_shares
        
    def buy_how_many(self, cur_date, cur_price):
        if(self.policy_loader.enable_one_account_position()==False):
            from_space_shares = self.get_shares_only_care_about_space(cur_date,cur_price)
            return from_space_shares
        else:
            from_loss_shares = self.get_shares_only_care_loss_money(cur_price)
            from_space_shares = self.get_shares_only_care_about_space(cur_date,cur_price)

            return min(from_loss_shares,from_space_shares)
    
    
#   根据入场后的涨幅来确定止损的位置
    def get_over_stop_loss_price(self, cur_date,increase_threshold=10):
        increase_per = self.account.increase_per[self.symbol]
        
        if(increase_per>increase_threshold):
            return self.account.get_last_time_buy_price(self.symbol)
        
        loss_percent = self.get_fix_stop_loss_percent()
        
        last_buy_price = self.account.get_last_time_buy_price(self.symbol)
        if(last_buy_price>0):
            stop_loss_price=last_buy_price*(1-loss_percent/100)
        else:
            stop_loss_price=last_buy_price*(1+loss_percent/100)

        return stop_loss_price
    
        